# Google Colab에서 Llama 3.2 finetuning

이 노트북은 Google Colab T4 GPU 환경에서 사용하는 것을 전제로 합니다.

In [1]:
!pip install -q accelerate peft bitsandbytes transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.9/310.9 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
!pip install -q --upgrade transformers trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.7 MB/s eta 0:00:00


In [3]:
!pip install -q accelerate>=0.26.0

In [4]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [5]:
# The model that you want to train from the Hugging Face hub
model_name = "Bllossom/llama-3.2-Korean-Bllossom-3B"

# The instruction dataset to use
#dataset_name = "rssaem/llama_bts_ko"
dataset_name = "rssaem/llm_ko_politec"

# Fine-tuned model name
#new_model = "llama-3.2-3b-bts"
new_model = "llama-3.2-3b-politec"


In [6]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 100

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001
# Optimizer to use
optim = "paged_adamw_32bit"
# Learning rate schedule
lr_scheduler_type = "cosine"
# Number of training steps (overrides num_train_epochs)
max_steps = -1
# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03
# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True
# Save checkpoint every X updates steps
save_steps = 0
# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None
# Pack multiple short examples in the same input sequence to increase efficiency
packing = False
# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

dataset[7]


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/474 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/33 [00:00<?, ? examples/s]

{'instruction': '폴리텍대학 광주캠퍼스 AI 융합과의 입학 조건은 무엇인가요?',
 'output': 'AI 융합과에 입학하기 위해서는 일반적으로 고등학교 졸업 이상의 학력이 요구됩니다. 또한, 관련 분야에 대한 관심과 기초적인 컴퓨터 활용 능력이 있으면 유리합니다. 각 학기마다 입학 전형이 다를 수 있으므로, 입학 전형에 대한 세부 사항은 대학 홈페이지나 입학처에서 확인하는 것이 좋습니다.',
 'Unnamed: 2': None,
 '__index_level_0__': 7,
 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n        You are a helpful assistant<|eot_id|>\n<|start_header_id|>user<|end_header_id|>\n\n        폴리텍대학 광주캠퍼스 AI 융합과의 입학 조건은 무엇인가요?<|eot_id|>\n<|start_header_id|>assistant<|end_header_id|>\n\n        AI 융합과에 입학하기 위해서는 일반적으로 고등학교 졸업 이상의 학력이 요구됩니다. 또한, 관련 분야에 대한 관심과 기초적인 컴퓨터 활용 능력이 있으면 유리합니다. 각 학기마다 입학 전형이 다를 수 있으므로, 입학 전형에 대한 세부 사항은 대학 홈페이지나 입학처에서 확인하는 것이 좋습니다.<|eot_id|>'}

In [8]:
def create_text_column(example):
    # 'text' 컬럼 생성
    text = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    example["text"] = text
    return example

# 'text' 컬럼 생성
datasetrs = dataset.map(create_text_column)

Map:   0%|          | 0/33 [00:00<?, ? examples/s]

In [9]:
datasetrs[7]

{'instruction': '폴리텍대학 광주캠퍼스 AI 융합과의 입학 조건은 무엇인가요?',
 'output': 'AI 융합과에 입학하기 위해서는 일반적으로 고등학교 졸업 이상의 학력이 요구됩니다. 또한, 관련 분야에 대한 관심과 기초적인 컴퓨터 활용 능력이 있으면 유리합니다. 각 학기마다 입학 전형이 다를 수 있으므로, 입학 전형에 대한 세부 사항은 대학 홈페이지나 입학처에서 확인하는 것이 좋습니다.',
 'Unnamed: 2': None,
 '__index_level_0__': 7,
 'text': '### Instruction:\n폴리텍대학 광주캠퍼스 AI 융합과의 입학 조건은 무엇인가요?\n\n### Response:\nAI 융합과에 입학하기 위해서는 일반적으로 고등학교 졸업 이상의 학력이 요구됩니다. 또한, 관련 분야에 대한 관심과 기초적인 컴퓨터 활용 능력이 있으면 유리합니다. 각 학기마다 입학 전형이 다를 수 있으므로, 입학 전형에 대한 세부 사항은 대학 홈페이지나 입학처에서 확인하는 것이 좋습니다.'}

Load tokenizer and model with QLoRA configuration

In [10]:
# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


Load base model

In [11]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto",
)

# model = AutoModelForCausalLM.from_pretrained(
#     model_name,
#     quantization_config=bnb_config,
#     device_map=device_map
# )
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)



config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [12]:

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=datasetrs,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:309: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/33 [00:00<?, ? examples/s]

Step,Training Loss
25,3.206100
50,1.787700
75,1.235200
100,0.837100
125,0.560400
150,0.407800
175,0.368600
200,0.296700
225,0.272200
250,0.237400


TrainOutput(global_step=900, training_loss=0.33708455562591555, metrics={'train_runtime': 1477.9175, 'train_samples_per_second': 2.233, 'train_steps_per_second': 0.609, 'total_flos': 5184892091394048.0, 'train_loss': 0.33708455562591555, 'epoch': 100.0})

In [13]:
# Save trained model
trainer.model.save_pretrained(new_model)

모델 테스트

In [14]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
#prompt = "BTS 음악의 컨셉은 무엇입니까?"
prompt = "폴리텍대학 광주캠퍼스 AI융합과에서 실습은 어떻게 이루어지나요?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

<s>[INST] 폴리텍대학 광주캠퍼스 AI융합과에서 실습은 어떻게 이루어지나요? [/INST] 폴리텍대학 광주캠퍼스 AI융합과에서는 실습으로 구성되어 있습니다. 이를 위해 어떤 활동이 이루어지나요? [/s[INST] 폴리텍대학 광주캠퍼스 AI융합과에서 실습은 어떻게 이루어지나요?를 cómo를 봐야 하는가요? 

 Antworten: AI융합과에서는 실습으로 구성되어 있습니다. 이를 위해 자주 교과 과목으로 구성되어 있습니다. 예를 들어, 데이터 분석, 머신러닝, 컴퓨터 비전, 자연어 처리, 웹 개발 등 자주 교과 과목에서 실습을 이루고 있습니다. 또한, 실습으로의 활동을 통해 학생들은 실무에 필요한 경험을 쌓을 수 있습니다. 따라서, 실습 활동을


In [15]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [16]:
%cd /content/gdrive/MyDrive/LLMClass/

/content/gdrive/MyDrive/LLMClass


In [17]:
#만약 finetuning 폴더가 없다면 생성
!mkdir finetuning

mkdir: cannot create directory ‘finetuning’: File exists


모델 저장

In [18]:
#savePath = "finetuning/llama3_2_rs_bts_200"
savePath = "finetuning/llama3_2_politec_200"
trainer.save_model(savePath)

In [19]:
def generate_response(prompt, model):
    encoded_input = tokenizer(prompt,  return_tensors="pt", add_special_tokens=True)
    model_inputs = encoded_input.to('cuda')

    generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=tokenizer.eos_token_id)
    # generated_ids = model.generate(**model_inputs, max_new_tokens=200, do_sample=True, pad_token_id=128009)

    decoded_output = tokenizer.batch_decode(generated_ids)

    return decoded_output[0].replace(prompt, "")
#key="BTS 음악의 컨셉은 무엇입니까?"
key="폴리텍대학 광주캠퍼스 AI융합과에서 실습은 어떻게 이루어지나요?"
prompt=f"""you are a assistant please answer in korean lanauage

### Instruction:
{key}

### Response:"""
generate_response(prompt, model)

"<|begin_of_text|> 한국에서는 교육 hệ계가 다양한데요, elementary부부터 중학교까지는 교육 기초를 다룹니다. 그러나 학습자들의 인강화된 융합 기술은 중### 중인 AI에서는 đặc사 실습이 가능합니다. 이를 위해 mỗi 수업마다 '사제 실습'로หมาย하는 것은 자격이 있습니다. 이를 통해 학습자들은 이 kinds of '사전 인강화의 사례 활용 및 실습 부문에서 많은 기회를 얻을 수 있습니다. 또한, 실습진습에서 중요한 것은 '사전 인강화'의 분야마다 different '사례 활용 및 실습'로หมาย합니다. 예를###로 보관하는 AI 융합 기술은 다양한 실습진습에서 '사전 인강화'를 포함해야 합니다. 이를 통해 학습기회가 있지만, 또한 실습진습을 ###로 보관하는 분야를 확인해야 합니다. 예"